
#LightGBM公式
<https://lightgbm.readthedocs.io/en/latest/>

#Sishihara氏のチュートリアル

<https://www.kaggle.com/sishihara/upura-kaggle-tutorial-04-lightgbm>




下から download-all をクリックしてtitanicのデータをダウンロード。

<https://www.kaggle.com/c/titanic/rules>


titanic.zipを解答してコードと同じフォルダに配置。

注意：colaboratoryならセッションごとにファイルをアップロードする。



In [11]:
# 特徴量の準備

import numpy as np
import pandas as pd

train = pd.read_csv("./titanic/train.csv")
test = pd.read_csv("./titanic/test.csv")
gender_submission = pd.read_csv("./titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [12]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0
4,5,0.0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1


In [13]:
# 不要データの削除
delete_columns = ['Name', 'PassengerId','Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

# trainとtestに再分離
train = data[:len(train)]
test = data[len(train):]

# 目的変数と特徴量に分離
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

# LightBGM用の下準備
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
categorical_features = ['Embarked', 'Pclass', 'Sex']



In [15]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
231,3,0,29.0,0,0,7.7750,0,1,1
836,3,0,21.0,0,0,8.6625,0,1,1
639,3,0,28.0,1,0,16.1000,0,2,0
389,2,1,17.0,0,0,12.0000,1,1,1
597,3,0,49.0,0,0,0.0000,0,1,1


In [14]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

Training until validation scores don't improve for 10 rounds.
[10]	training's binary_logloss: 0.425241	valid_1's binary_logloss: 0.478975
[20]	training's binary_logloss: 0.344972	valid_1's binary_logloss: 0.444039
[30]	training's binary_logloss: 0.301357	valid_1's binary_logloss: 0.436304
[40]	training's binary_logloss: 0.265535	valid_1's binary_logloss: 0.438139
Early stopping, best iteration is:
[38]	training's binary_logloss: 0.271328	valid_1's binary_logloss: 0.435633


/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


In [16]:
y_pred[:10]

array([0.0320592 , 0.34308916, 0.09903007, 0.05723199, 0.39919906,
       0.22299318, 0.55036246, 0.0908458 , 0.78109016, 0.01881392])

In [17]:
# 0.5以上を1（生存）とみなす
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [19]:
# 提出データの作成
sub = gender_submission
sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm.csv", index=False)

sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
